<a href="https://colab.research.google.com/github/AiDarkEzio/Project-Jupyter-NoteBooks/blob/main/GDrive_Version___InvokeAI_Web_UI_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # Installing InvokeAI

#@markdown Use Google Drive to store models (uses about 13 GB). Uncheck this if you don't have enough space in your Drive.
useGoogleDrive = True #@param {type:"boolean"}

googleDriveModelsFolder = '/stablemodels' #@param {type:"string"}

#@markdown This step usually takes about 5 minutes.

#@markdown You can ignore the message about restarting the runtime.
import os
import subprocess
from google.colab import drive
if useGoogleDrive:
  drive.mount('/content/drive')
  if not googleDriveModelsFolder.startswith('/'):
    googleDriveModelsFolder = '/' + googleDriveModelsFolder
  modelsPath = "/content/drive/MyDrive"+googleDriveModelsFolder
  if not modelsPath.endswith("/"):
   modelsPath = modelsPath + "/"

env = os.environ.copy()

!git clone -b main https://github.com/invoke-ai/InvokeAI
#!git clone --depth 1 --branch 3.0.2rc1 https://github.com/invoke-ai/InvokeAI

!pip install pillow==9.5.0 requests==2.31.0 xformers==0.0.20 triton==2.0.0
!pip install git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install git+https://github.com/Birch-san/k-diffusion.git@mps#egg=k-diffusion
!pip install git+https://github.com/invoke-ai/clipseg.git@relaxed-python-requirement#egg=clipseg
!pip install git+https://github.com/invoke-ai/PyPatchMatch@0.1.4#egg=pypatchmatch
%cd /content/InvokeAI/
!pip install -q -e .

In [ ]:
!mkdir /content/InvokeAI/invokeai
!mkdir /content/InvokeAI/invokeai/configs

#@markdown Download only the default model in initial configuration.
#@markdown Checking this prevents running out of space in Colab.

defaultOnly = True #@param {type:"boolean"}
skipWeights = True #@param {type:"boolean"}

#@markdown This step usually takes about 2 minutes with only the default model and no weights.

#@markdown You can ignore "File exists" warnings in the output.

cmd = 'invokeai-configure --root_dir /content/InvokeAI/invokeai --yes'

if defaultOnly:
  cmd += ' --default_only'

if skipWeights:
  cmd += ' --skip-sd-weights'

subprocess.run(cmd, shell=True, env=env)

In [ ]:
#@markdown Adding the SDXL Base Model

#@markdown Installing SDXL base took about 20 minutes initially, but it's finished instantly
#@markdown in subsequent runs if Google Drive is enabled. You can execute the first run in a runtime
#@markdown without a GPU to save the model to Google Drive without spending GPU time.

import os.path
from os import path

# Install the SDXL base model
def installSdxl(env):
  installCmd = 'invokeai-model-install --add "stabilityai/stable-diffusion-xl-base-1.0" --root_dir /content/InvokeAI/invokeai'
  subprocess.run(installCmd, shell=True, env=env)

sdxlBaseSubfolderName = '/stable-diffusion-xl-base-1-0'

if useGoogleDrive:
  alreadyInstalled = True

  driveSdxlMainFolder = modelsPath + "sdxl/main"
  if not path.exists(driveSdxlMainFolder):
    os.makedirs(driveSdxlMainFolder, exist_ok=True)
    alreadyInstalled = False

  localModelsSdxlFolder = "/content/InvokeAI/invokeai/models/sdxl/"
  localSdxlMainFolder = localModelsSdxlFolder + "main"

  subprocess.run('rm -rf ' + localModelsSdxlFolder, shell=True, env=env)
  subprocess.run('rmdir ' + localModelsSdxlFolder, shell=True, env=env)

  if not alreadyInstalled:
    if not path.exists(localModelsSdxlFolder):
      os.makedirs(localModelsSdxlFolder, exist_ok=True)
    subprocess.run('ln -s '+driveSdxlMainFolder+' '+localModelsSdxlFolder, shell=True, env=env)
    installSdxl(env)
  else:
    if not path.exists(localSdxlMainFolder):
      os.makedirs(localSdxlMainFolder, exist_ok=True)
    subprocess.run('ln -s '+driveSdxlMainFolder + sdxlBaseSubfolderName+' '+ localSdxlMainFolder, shell=True, env=env)
    updateModelsYaml = True
    with open('/content/InvokeAI/invokeai/configs/models.yaml') as f:
      if 'stable-diffusion-xl-base-1-0' in f.read():
        updateModelsYaml = False
    if updateModelsYaml:
      with open('/content/InvokeAI/invokeai/configs/models.yaml', 'a') as file:
        lines = [
          'sdxl/main/stable-diffusion-xl-base-1-0:\n',
          '  path: sdxl/main/stable-diffusion-xl-base-1-0\n',
          '  description: Stable Diffusion XL base model (12 GB)\n',
          '  variant: normal\n',
          '  format: diffusers\n'
        ]
        file.writelines(lines)
else:
  installSdxl(env)

In [ ]:
#@markdown Adding the refiner and vae. This one took about 14 minutes.
#@markdown Skip this step if you don't need these models.
!invokeai-model-install --add "stabilityai/stable-diffusion-xl-refiner-1.0" --root_dir /content/InvokeAI/invokeai --yes
!invokeai-model-install --add "madebyollin/sdxl-vae-fp16-fix" --root_dir /content/InvokeAI/invokeai --yes

In [ ]:
#@markdown # Option 1: Starting the Web UI with ngrok
!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys

Ngrok_token = "" #@param {type:"string"}
#@markdown - Add ngrok token (obtainable from https://ngrok.com)

share=''
if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(9090 , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True).public_url
  print(srv)
  !python /content/InvokeAI/scripts/invokeai-web.py --root /content/InvokeAI/invokeai/
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')


In [ ]:
#@markdown # Option 2: Starting the Web UI with Localtunnel

%cd /content/InvokeAI/invokeai/
!npm install -g localtunnel

#@markdown Copy the IP address shown in the output above the line
#@markdown "your url is: https://some-random-words.loca.lt"
!wget -q -O - ipv4.icanhazip.com

#@markdown Wait for the line that says "Uvicorn running on http://127.0.0.1:9090 (Press CTRL+C to quit)"

#@markdown Click the localtunnel url and paste the IP you copied earlier to the "Endpoint IP" text field
!lt --port 9090 --local_https False & python /content/InvokeAI/scripts/invokeai-web.py --root /content/InvokeAI/invokeai/

#@markdown If the UI shows a red dot that says 'disconnected' when hovered in the upper
#@markdown right corner and the Invoke button is disabled, change 'https' to 'http'
#@markdown in the browser's address bar and press enter.
#@markdown When the page reloads, the UI should work properly.
